In [23]:
import numpy as np
import pandas as pd
from stockstats import StockDataFrame as sd_frame

### Preprocessing the Dow 30 Stock Data
- Transform the data format for trading
- Add the technical Indicatiors: MACD, RSI, CCI and ADX
- Add The Systemic Risk Indicator: Turbulence Index 

In [24]:
DATA_PATH = 'data/dow30_2009_2020.csv'

In [25]:
stock_df = pd.read_csv(DATA_PATH)

In [26]:
stock_df.head()

,Unnamed: 0,datadate,tic,prccd,ajexdi,prcod,prchd,prcld,cshtrd
0,0,20090102,AXP,19.33,1.0,18.57,19.52,18.40,10955620.0
1,1,20090105,AXP,19.95,1.0,19.20,20.24,19.00,16019130.0
2,2,20090106,AXP,21.07,1.0,20.30,21.38,20.01,13819380.0
3,3,20090107,AXP,20.01,1.0,20.53,20.72,19.83,15699820.0
4,4,20090108,AXP,20.04,1.0,19.80,20.17,19.40,12255070.0


In [27]:
# Training Data starts from 2009/01/01
stock_df = stock_df[stock_df.datadate >= 20090000]

#### 1. Caculate the stock prices

- The  stock  data  set  will  use  the  Dow  30  historical  data.
- The data elements include the daily Open(prcod), High(prchd),Low(prcld) and Close Prices(prccd); the daily volume for com-mon  share  traded(cshtrd),  the  Cumulative  Adjustment  Factor for the stock (AJEXDI). 
- The Adjusted Closing Price willbe used for the stock price which is computed by PRCCD/AJEXDI.

In [28]:
stock_dataset = stock_df.copy()
stock_dataset = stock_dataset[['datadate', 'tic', 'prccd', 'ajexdi', 'prcod', 'prchd', 'prcld', 'cshtrd']]
stock_dataset.head()

,datadate,tic,prccd,ajexdi,prcod,prchd,prcld,cshtrd
0,20090102,AXP,19.33,1.0,18.57,19.52,18.40,10955620.0
1,20090105,AXP,19.95,1.0,19.20,20.24,19.00,16019130.0
2,20090106,AXP,21.07,1.0,20.30,21.38,20.01,13819380.0
3,20090107,AXP,20.01,1.0,20.53,20.72,19.83,15699820.0
4,20090108,AXP,20.04,1.0,19.80,20.17,19.40,12255070.0


- Adjusted close price (Close) = PRCCD/AJEXDI 
- Open =  PRCOD/AJEXDI
- High =  PRCHD/AJEXDI
- Low =   PRCLD/AJEXDI
- Volume = CSHTRD

In [29]:
stock_dataset['ajexdi'] = stock_dataset['ajexdi'].apply(lambda x: 1 if x == 0 else x)
stock_dataset['close'] = stock_dataset['prccd'] / stock_dataset['ajexdi']


stock_dataset['open'] = stock_dataset['prcod'] / stock_dataset['ajexdi']
stock_dataset['high'] = stock_dataset['prchd'] / stock_dataset['ajexdi']
stock_dataset['low'] = stock_dataset['prcld'] / stock_dataset['ajexdi']
stock_dataset['volume'] = stock_dataset['cshtrd']

stock_dataset = stock_dataset[['datadate', 'tic', 'close', 'open', 'high', 'low', 'volume']]
stock_dataset = stock_dataset.sort_values(['tic', 'datadate'], ignore_index=True)

In [30]:
stock_dataset.head()

,datadate,tic,close,open,high,low,volume
0,20090102,AAPL,12.964286,12.268571,13.005714,12.165714,26641980.0
1,20090105,AAPL,13.511429,13.310000,13.740000,13.244286,42187570.0
2,20090106,AAPL,13.288571,13.707143,13.881429,13.198571,46038530.0
3,20090107,AAPL,13.001429,13.115714,13.214286,12.894286,26890770.0
4,20090108,AAPL,13.242857,12.918571,13.307143,12.862857,23670270.0


#### 2. Add Technical Indicators:
- Moving Average Convergence Diver-gence.(MACD)
- Relative  Strength  Index(RSI)
- AverageDirection Index(ADI)
- Average Directional MovementIndex(ADX)

In [31]:
# StockDataFrame with the retype function which convert a pandas.DataFrame to a StockDataFrame

stock_dataset_ind = sd_frame.retype(stock_dataset.copy())

In [32]:
stock_dataset_ind.head()

,datadate,tic,close,open,high,low,volume
0,20090102,AAPL,12.964286,12.268571,13.005714,12.165714,26641980.0
1,20090105,AAPL,13.511429,13.310000,13.740000,13.244286,42187570.0
2,20090106,AAPL,13.288571,13.707143,13.881429,13.198571,46038530.0
3,20090107,AAPL,13.001429,13.115714,13.214286,12.894286,26890770.0
4,20090108,AAPL,13.242857,12.918571,13.307143,12.862857,23670270.0


In [36]:
dow30_tics = stock_dataset_ind.tic.unique()

In [37]:
dow30_tics

array(['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DD', 'DIS', 'GS', 'HD',
       'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT',
       'NKE', 'PFE', 'PG', 'RTX', 'TRV', 'UNH', 'V', 'VZ', 'WBA', 'WMT',
       'XOM'], dtype=object)

In [39]:
MACD = pd.DataFrame()
RSI = pd.DataFrame()
CCI = pd.DataFrame()
ADX = pd.DataFrame()

for i in range(len(dow30_tics)):
    
    MACD = MACD.append(pd.DataFrame(stock_dataset_ind[stock_dataset_ind.tic == dow30_tics[i]]['macd']), ignore_index=True)
   
    RSI = RSI.append(pd.DataFrame(stock_dataset_ind[stock_dataset_ind.tic == dow30_tics[i]]['rsi_30']), ignore_index=True)

    CCI = CCI.append(pd.DataFrame(stock_dataset_ind[stock_dataset_ind.tic == dow30_tics[i]]['cci_30']), ignore_index=True)

    ADX = ADX.append(pd.DataFrame(stock_dataset_ind[stock_dataset_ind.tic == dow30_tics[i]]['dx_30']), ignore_index=True)


stock_dataset['macd'] = MACD
stock_dataset['rsi'] = RSI
stock_dataset['cci'] = CCI
stock_dataset['adx'] = ADX


In [40]:
stock_dataset.head()

,datadate,tic,close,open,high,low,volume,macd,rsi,cci,adx
0,20090102,AAPL,12.964286,12.268571,13.005714,12.165714,26641980.0,0.000000,NaN,NaN,NaN
1,20090105,AAPL,13.511429,13.310000,13.740000,13.244286,42187570.0,0.012276,100.000000,66.666667,100.000000
2,20090106,AAPL,13.288571,13.707143,13.881429,13.198571,46038530.0,0.008731,70.355356,45.847589,100.000000
3,20090107,AAPL,13.001429,13.115714,13.214286,12.894286,26890770.0,-0.003463,50.429216,-30.767206,43.608258
4,20090108,AAPL,13.242857,12.918571,13.307143,12.862857,23670270.0,-0.000408,60.227007,-8.239643,48.358160


In [41]:
stock_dataset.fillna(method='bfill',inplace=True)
stock_dataset.head()

,datadate,tic,close,open,high,low,volume,macd,rsi,cci,adx
0,20090102,AAPL,12.964286,12.268571,13.005714,12.165714,26641980.0,0.000000,100.000000,66.666667,100.000000
1,20090105,AAPL,13.511429,13.310000,13.740000,13.244286,42187570.0,0.012276,100.000000,66.666667,100.000000
2,20090106,AAPL,13.288571,13.707143,13.881429,13.198571,46038530.0,0.008731,70.355356,45.847589,100.000000
3,20090107,AAPL,13.001429,13.115714,13.214286,12.894286,26890770.0,-0.003463,50.429216,-30.767206,43.608258
4,20090108,AAPL,13.242857,12.918571,13.307143,12.862857,23670270.0,-0.000408,60.227007,-8.239643,48.358160


#### 3. Add the System Risk Indicator: Turbulence Index
- Kritzman and Li (2010) present “a mathematical measure of financial turbulence” based on the Mahalanobis Distance.
- Qualitatively: financial turbulence is a condition where Asset prices move by an uncharacteristically large amount. Asset prices movements violate the existing correlation structure (the “decoupling of correlated assets” and the “convergence of uncorrelated assets”).
- Use cases for Financial Turbulence include stress-testing investment portfolios, building turbulence-resistant investment portfolios, and scaling exposure to risk.

![img](turbulence.jpg)





In [42]:
df = stock_dataset
df_price_pivot=df.pivot(index='datadate', columns='tic', values='close')
unique_date = df.datadate.unique()

In [43]:
df_price_pivot.head()

tic,AAPL,AXP,BA,CAT,CSCO,CVX,DD,DIS,GS,HD,...,PFE,PG,RTX,TRV,UNH,V,VZ,WBA,WMT,XOM
datadate,,,,,,,,,,,,,,,,,,,,,
20090102,12.964286,19.33,45.25,46.91,16.96,76.52,46.234623,23.92,86.76,24.13,...,18.27,62.80,54.95,45.20,27.59,13.3600,34.64,25.55,57.18,81.64
20090105,13.511429,19.95,46.17,46.08,17.11,76.66,45.154515,23.50,88.78,24.71,...,18.16,62.35,54.30,44.53,27.14,13.4550,32.48,26.84,56.52,81.63
20090106,13.288571,21.07,46.31,45.80,17.79,77.35,48.154815,24.31,88.71,25.26,...,17.80,62.17,54.88,43.17,26.50,14.4025,31.96,26.71,56.02,80.30
20090107,13.001429,20.01,44.76,43.67,17.32,73.96,48.244824,23.18,84.50,24.57,...,17.49,61.08,53.52,41.46,26.30,14.0900,31.90,27.03,55.54,78.25
20090108,13.242857,20.04,44.79,44.02,17.54,74.24,47.434743,22.90,85.41,24.39,...,17.65,60.41,53.51,42.23,26.71,13.9400,32.39,26.84,51.38,79.09


In [44]:
# The NYSE and NASDAQ average about 253 trading days a year.
start = 253
turbulence_index = [0]*start
count=0

for i in range(start-1,len(unique_date)):
    current_price = df_price_pivot[df_price_pivot.index == unique_date[i]]
    hist_price = df_price_pivot[[n in unique_date[0:i] for n in df_price_pivot.index ]]
    cov_temp = hist_price.cov()
    current_temp=(current_price - np.mean(hist_price,axis=0))
    temp = current_temp.values.dot(np.linalg.inv(cov_temp)).dot(current_temp.values.T)
    if temp>0:
        count+=1
        if count>2:
            turbulence_temp = temp[0][0]
        else:
            turbulence_temp=0
    else:
        turbulence_temp=0
    turbulence_index.append(turbulence_temp)
    

In [45]:
turbulence_index = pd.DataFrame({'datadate':df_price_pivot.index,'turbulence':turbulence_index})
turbulence_index.head()

,datadate,turbulence
0,20090102,0.0
1,20090105,0.0
2,20090106,0.0
3,20090107,0.0
4,20090108,0.0


In [46]:
stock_dataset = stock_dataset.merge(turbulence_index, on='datadate')
stock_dataset = stock_dataset.sort_values(['datadate','tic']).reset_index(drop=True)
stock_dataset.head()

,datadate,tic,close,open,high,low,volume,macd,rsi,cci,adx,turbulence
0,20090102,AAPL,12.964286,12.268571,13.005714,12.165714,26641980.0,0.0,100.0,66.666667,100.0,0.0
1,20090102,AXP,19.330000,18.570000,19.520000,18.400000,10955620.0,0.0,100.0,66.666667,100.0,0.0
2,20090102,BA,45.250000,42.800000,45.560000,42.780000,7010171.0,0.0,100.0,66.666667,100.0,0.0
3,20090102,CAT,46.910000,44.910000,46.980000,44.710000,7116726.0,0.0,0.0,66.666667,100.0,0.0
4,20090102,CSCO,16.960000,16.410000,17.000000,16.250000,40977480.0,0.0,100.0,66.666667,100.0,0.0


In [47]:
stock_dataset.head(-10)

,datadate,tic,close,open,high,low,volume,macd,rsi,cci,adx,turbulence
0,20090102,AAPL,12.964286,12.268571,13.005714,12.165714,26641980.0,0.000000,100.000000,66.666667,100.000000,0.000000
1,20090102,AXP,19.330000,18.570000,19.520000,18.400000,10955620.0,0.000000,100.000000,66.666667,100.000000,0.000000
2,20090102,BA,45.250000,42.800000,45.560000,42.780000,7010171.0,0.000000,100.000000,66.666667,100.000000,0.000000
3,20090102,CAT,46.910000,44.910000,46.980000,44.710000,7116726.0,0.000000,0.000000,66.666667,100.000000,0.000000
4,20090102,CSCO,16.960000,16.410000,17.000000,16.250000,40977480.0,0.000000,100.000000,66.666667,100.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
87765,20200817,MCD,208.670000,206.940000,209.500000,206.850000,2616317.0,4.437168,62.865408,137.518730,45.783915,127.002248
87766,20200817,MMM,164.710000,165.170000,166.370000,163.650000,2066626.0,2.225043,56.373586,128.880338,28.124009,127.002248
87767,20200817,MRK,84.760000,84.280000,85.180000,84.010000,8358406.0,1.317735,59.504141,188.258240,36.218432,127.002248
87768,20200817,MSFT,210.280000,209.600000,211.187400,208.915000,20156790.0,1.638018,55.398181,34.180601,13.509338,127.002248


In [48]:
stock_dataset.to_csv('dow30_dataset.csv')

In [51]:
print(stock_dataset.size)

1053360


In [52]:
unique_trade_date = stock_dataset[(stock_dataset.datadate > 20151001)&(stock_dataset.datadate <= 20200101)].datadate.unique()
print(unique_trade_date)


[20151002 20151005 20151006 ... 20191227 20191230 20191231]
